#  TSP using A* search

# Heuristic: Minimum Spanning Tree.

In [71]:
from collections import defaultdict
import math
import numpy as np
import heapq
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [72]:
class Graph: # Class describing graph

    def __init__(self,vertices):
        self.nodes = [] #list of nodes
        self.edges = [] #Graph 
        for i in range(len(vertices)):
            self.nodes.append(vertices[i])
        self.child = defaultdict(list)

    def addEdge(self,u,v,w): 
        for edges in self.edges:
            if u==edges[0] and v==edges[1]:
                print("Edge already exists")
                return
        self.edges.append([u,v,w])
        self.child[u].append((v, w))

    def Cost(self,nodes):
        if len(nodes)<=1:
            return 0
        else:
            total_cost=0
            i=1
            while i<len(nodes):
                for edge in self.edges:
                    if nodes[i-1]==edge[0] and nodes[i]==edge[1]:
                        total_cost=total_cost+edge[2]
                i=i+1
            for edge in self.edges:
                    if nodes[0]==edge[0] and nodes[len(nodes) - 1]==edge[1]:
                        total_cost=total_cost+edge[2]
            return total_cost
    
    # function that run the "A*" algorithm
    def astar(self, source_node):
        if not self.edges:
            print('Error: There are no edges in graph!')
        else:
            print("Source node:",source_node)
            if source_node in self.nodes:
                queue = PriorityQueue()
                iteration=1 #gives number of iterations
                g_cost, h_cost = 0, self.MST(source_node)
                f_cost = g_cost + h_cost
                queue.insert((source_node, g_cost, h_cost), f_cost)
                # visited keeps the track of order of visited nodes to the path
                visited = []
                visited.append(source_node)
                while len(visited) < len(self.nodes):
                    # check if children there is any node remained to expand while goal state is not reached
                    if queue.isEmpty():
                        print("Error : Goal state cannot be reached. Graph is not connected")
                        return 0,[]
                    current_node, g_cost, h_cost = queue.remove()
                    # remaining_nodes calculated unexpanded neighbours for current_node
                    # remaining_nodes functions as fringe_list
                    remaining_nodes = []
                    # gets all the child of "current_node"
                    child = self.child[current_node]
                    minimum = math.inf # initialising to infinity
                    a,b,c,d=0,0,0,0
                    temp_child=[]
                    for successor in child:
                        if successor[0] not in visited:
                            temp_child.append(successor)
                    if len(temp_child)!= 0:
                        for successor in temp_child:
                            # empty remaining_nodes to remove previous nodes that could have been already visited
                            remaining_nodes.clear()
                            destination, weight = successor
                            # add unvisited nodes to remaining_nodes
                            for i in self.nodes:
                                if i not in visited:
                                    remaining_nodes.append(i)
                            # if remaining node contains a single unvisited node, h_cost = 0 for that single node
                            if len(remaining_nodes)==1:
                                new_g_cost = g_cost + weight
                                h_cost = 0
                                f_cost = new_g_cost + h_cost
                                a,b,c,d=destination,f_cost,new_g_cost,h_cost
                            else:
                                # create a temorary graph using nodes from remaining_nodes 
                                graph_prime = Graph(remaining_nodes)
                                # add edges to temporary graph if remaining_nodes exist in original graph 
                                for edge in self.edges:
                                    if edge[0] in remaining_nodes and edge[1] in remaining_nodes:
                                        graph_prime.addEdge(edge[0],edge[1],edge[2]) 
                                # calculates costs for temporary graph
                                new_g_cost = g_cost + weight
                                h_cost = graph_prime.MST(destination)
                                f_cost = new_g_cost + h_cost
                                # choose the minimum f_cost of all the child
                                if f_cost < minimum :
                                    minimum = f_cost
                                    a,b,c,d=destination,f_cost,new_g_cost,h_cost
                        destination=a
                        f_cost=b
                        new_g_cost=c
                        h_cost=d
                        # add this minimum node to visited whuch represents our path
                        visited.append(destination)
                        # add the minimum successor to queue
                        queue.insert((destination, new_g_cost, h_cost), f_cost)
                        iteration = iteration + 1
                        # verifies that reached the goal
                        if len(visited) == len(self.nodes):
                            print("\nGoal state reached\n")

                if len(visited) == len(self.nodes): 
                    return visited
            else:
                print('Error: the node(s) not exists in the graph!!')

    def MST(self,source):
        priority_queue = { source : 0 }
        mstCost = 0
        added = [False] * len(self.edges)

        while priority_queue :
            # Choose the adjacent node with the least edge cost
            node = min(priority_queue, key=priority_queue.get)
            cost = priority_queue[node]
            
            del priority_queue[node] #removing node from queue
            if node < len(added) and added[node] == False :
                mstCost += cost
                added[node] = True
                for item in self.child[node] :
                    node_prime = item[0]
                    cost_prime = item[1]
                    if node_prime < len(added) and added[node_prime] == False :
                        priority_queue[node_prime] = cost_prime

        return mstCost

In [73]:
class PriorityQueue:

    def __init__(self):
        self._queue = []
        self._index = 0

    def insert(self, item, priority):
        heapq.heappush(self._queue, (priority, self._index, item))
        self._index += 1

    def remove(self):
        return heapq.heappop(self._queue)[-1]

    def isEmpty(self):
        return len(self._queue) == 0

    def getSize(self):
        return self._index


In [74]:
def print_path(path):
    print("Path found")
    print()
    print("Path:")
    for i in range(len(path)-1):
        print(path[i],"->",path[i+1])
    print(path[len(path)-1],"->",path[0])

# TEST CASES:

In [75]:
g = Graph([0,1,2,3,4])

g.addEdge(0, 1, 2) 
g.addEdge(1, 0, 2)

g.addEdge(0,2,3)
g.addEdge(2,0,3)

g.addEdge(0, 3, 6) 
g.addEdge(3, 0, 6)

g.addEdge(0, 4, 7) 
g.addEdge(4, 0, 7)

g.addEdge(1, 2, 3) 
g.addEdge(2, 1, 3)

g.addEdge(1, 3, 8) 
g.addEdge(3, 1, 8)
 
g.addEdge(1, 4, 5) 
g.addEdge(4, 1, 5)

g.addEdge(3, 2, 8) 
g.addEdge(2, 3, 8)

g.addEdge(4, 2, 7) 
g.addEdge(2, 4, 7)

g.addEdge(3, 4, 9) 
g.addEdge(4, 3, 9)

path = g.astar(0) # executes the algorithm
total_cost = g.Cost(path)
if total_cost:
    print_path(path)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Source node: 0

Goal state reached

Path found

Path:
0 -> 2
2 -> 1
1 -> 4
4 -> 3
3 -> 0
total_cost 26


In [76]:
g = Graph([0,1,2,3,4]) 
g.addEdge(0, 1, 2) 
g.addEdge(1, 0, 2)

g.addEdge(0, 3, 6) 
g.addEdge(3, 0, 6)

g.addEdge(1, 2, 3) 
g.addEdge(2, 1, 3)

g.addEdge(1, 3, 8) 
g.addEdge(3, 1, 8)
 
g.addEdge(1, 4, 5) 
g.addEdge(4, 1, 5)

g.addEdge(4, 2, 7) 
g.addEdge(2, 4, 7)

g.addEdge(3, 4, 9) 
g.addEdge(4, 3, 9)



path = g.astar(0) # executes the algorithm
total_cost = g.Cost(path)
if total_cost:
    print_path(path)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Source node: 0

Goal state reached

Path found

Path:
0 -> 1
1 -> 2
2 -> 4
4 -> 3
3 -> 0
total_cost 27


In [77]:
nodes=[0,1,2,3,4,5,6,7,8,9]
g = Graph(nodes)
for i in range(10):
    for j in range(10):
        if i>j:
            weight = np.random.randint(20)
            g.addEdge(i,j,weight)
            g.addEdge(j,i,weight)
path = g.astar(0) # executes the algorithm
total_cost = g.Cost(path)
if total_cost:
    print_path(path)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Source node: 0

Goal state reached

Path found

Path:
0 -> 3
3 -> 7
7 -> 9
9 -> 8
8 -> 2
2 -> 5
5 -> 6
6 -> 1
1 -> 4
4 -> 0
total_cost 71


In [78]:
g=Graph([0,1,2,3])

g.addEdge(0, 1, 10) 
g.addEdge(1, 0, 10)

g.addEdge(0, 2, 15) 
g.addEdge(2, 0, 15)

g.addEdge(0, 3, 20) 
g.addEdge(3, 0, 20)

g.addEdge(1, 2, 35) 
g.addEdge(2, 1, 35)

g.addEdge(1, 3, 25) 
g.addEdge(3, 1, 25)

g.addEdge(3, 2, 30) 
g.addEdge(2, 3, 30)


path = g.astar(0) # executes the algorithm
total_cost = g.Cost(path)
if total_cost:
    print_path(path)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Source node: 0

Goal state reached

Path found

Path:
0 -> 1
1 -> 3
3 -> 2
2 -> 0
total_cost 80


In [79]:
g = Graph([0,1,2])
g.addEdge(0,1,1)
g.addEdge(1,0,1)

path = g.astar(0) # executes the algorithm
total_cost = g.Cost(path)
if total_cost:
    print_path(path)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Source node: 0
Error : Goal state cannot be reached. Graph is not connected
Did not reach the goal!


In [80]:
g = Graph([0,1,2,3,4]) 
g.addEdge(0, 1, 2) 
g.addEdge(1, 0, 2)

g.addEdge(0, 3, 6) 
g.addEdge(3, 0, 6)

g.addEdge(1, 2, 3) 
g.addEdge(2, 1, 3)

g.addEdge(1, 3, 8) 
g.addEdge(3, 1, 8)
 
g.addEdge(1, 4, 5)  
g.addEdge(4, 1, 5)

g.addEdge(4, 2, 7) 
g.addEdge(2, 4, 7)

g.addEdge(3, 4, 9) 
g.addEdge(4, 3, 9)



path = g.astar(4) # executes the algorithm
total_cost = g.Cost(path)
if total_cost:
    print_path(path)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Source node: 4
Error : Goal state cannot be reached. Graph is not connected
Did not reach the goal!


In [81]:
nodes=[]
for i in range(50):
    nodes.append(i)
g = Graph(nodes)
for i in range(50):
    for j in range(50):
        if i>j:
            weight = np.random.randint(20)
            g.addEdge(i,j,weight)
            g.addEdge(j,i,weight)
            
path = g.astar(1) # executes the algorithm
total_cost = g.Cost(path)
if total_cost:
    print_path(path)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Source node: 1

Goal state reached

Path found

Path:
1 -> 28
28 -> 9
9 -> 13
13 -> 7
7 -> 0
0 -> 11
11 -> 2
2 -> 17
17 -> 6
6 -> 10
10 -> 8
8 -> 14
14 -> 30
30 -> 21
21 -> 23
23 -> 45
45 -> 41
41 -> 37
37 -> 44
44 -> 25
25 -> 22
22 -> 15
15 -> 29
29 -> 40
40 -> 18
18 -> 24
24 -> 3
3 -> 20
20 -> 4
4 -> 26
26 -> 5
5 -> 27
27 -> 42
42 -> 31
31 -> 43
43 -> 16
16 -> 33
33 -> 49
49 -> 47
47 -> 12
12 -> 32
32 -> 19
19 -> 46
46 -> 35
35 -> 48
48 -> 38
38 -> 39
39 -> 34
34 -> 36
36 -> 1
total_cost 52
